In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax_smi
jax_smi.initialise_tracking()

In [14]:
from scripts.train_gpt2_sae import main
main(size=2, layer=16, push_to_hub=("nev/gpt2_medium_saes-saex-test", "l16-test"))

Loading model...


/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at openai-community/gpt2-large were not used when initializing FlaxGPT2LMHeadModel: {('h', '6', 'attn', 'bias'), ('h', '33', 'attn', 'bias'), ('h', '3', 'attn', 'bias'), ('h', '5', 'attn', 'bias'), ('h', '7', 'attn', 'bias'), ('h', '12', 'attn', 'bias'), ('h', '35', 'attn', 'bias'), ('h', '22', 'attn', 'bias'), ('h', '13', 'attn', 'bias'), ('h', '17', 'attn', 'bias'), ('h', '1', 'attn', 'bias'), ('h', '14', 'attn', 'bias'), ('h', '21', 'attn', 'bias'), ('h', '20', 'attn', 'bias'), ('h', '19', 'attn', 'bias'), ('h', '11', 'attn', 'bias'), ('h', '31', 'attn', 'bias'), ('h', '15', 'attn', 'bias'), ('h', '27', 'attn', 'bias

Creating SAE...
Loading dataset...


/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Creating buffer...
Training for 100000 iterations
Sparsity coefficient: 5e-05


Learning rate: 0.0004 warmed up for 128 iterations and cycled every 100000 iterations


  0%|          | 0/100000 [00:00<?, ?it/s]

In [3]:
from saex.iterable_dataset import IterableDatasetConfig
from saex.models.micrlhf_model import MicrlhfModelConfig
from saex.model_haver import ModelHaver
from saex.sae import SAEConfig
from more_itertools import chunked


n_features = 3072
batch_size = 64
sae_config = SAEConfig(
    n_dimensions=n_features,
    sparsity_loss_type="l1",
    sparsity_coefficient=0,
    batch_size=batch_size,
    expansion_factor=32,
    use_encoder_bias=True,
    remove_decoder_bias=False,
    encoder_init_method="orthogonal",
    decoder_init_method="pseudoinverse",
    decoder_bias_init_method="zeros",
    reconstruction_loss_type="mse_batchnorm",
    project_updates_from_dec=True,
    death_loss_type="dm_ghost_grads",
    death_penalty_threshold=5e-7,
    death_penalty_coefficient=0.25,
    dead_after=1_000,
    buffer_size=2_000,
    restrict_dec_norm="exact",
    sparsity_tracking_epsilon=0.1,
    is_gated=True,
)
dataset_config = IterableDatasetConfig(
    dataset_name="nev/openhermes-2.5-phi-format-text",
)
model_config = MicrlhfModelConfig(
    tokenizer_path="microsoft/Phi-3-mini-4k-instruct",
    gguf_path="weights/phi-3-16.gguf",
    device_map="tpu:0",
    use_flash=False,
    layer=11,
    max_seq_len=64,
)
haver = ModelHaver(model_config=model_config, sae_config=sae_config,
                    dataset_config=dataset_config,
                    sae_restore="weights/phi-l11-gated.safetensors")

/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading model...
Creating SAE...
Loading checkpoint (weights/phi-l11-gated.safetensors)...
Weights restored.
Loading dataset...


In [4]:
from collections import defaultdict
from tqdm.auto import tqdm
import jax.numpy as jnp
import numpy as np
import jax


tokens_processed = 0
activ_cache = defaultdict(list)
for texts in chunked(bar := tqdm(haver.create_dataset()), batch_size):
    activations, model_misc = haver.model(texts)
    mask = model_misc.get("mask")
    pre_relu, hiddens = haver.sae.encode(activations)
    bar.set_postfix(l0=((hiddens != 0).sum(-1) * mask).mean() / mask.mean())
    hiddens = np.asarray(hiddens.astype(jnp.float16))
    for i, h in enumerate(hiddens):
        if not mask[i]:
            continue
        active_features = np.nonzero(h)[0]
        feature_activations = h[active_features]
        for f, a in zip(active_features, feature_activations):
            activ_cache[int(f)].append((tokens_processed + i, float(a)))
    tokens_processed += hiddens.shape[0]

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [5]:
def visualize(feature, thresh=6.0):
    cache = activ_cache[feature]
    if not cache:    
        return
    tokens, activs = zip(*cache)
    if max(activs) < thresh:
        return
    freq = len(tokens) / tokens_processed
    print(freq)
    if freq > 0.03:
        return
    tokens_viewed = 0
    sli = 24
    for texts in chunked(tqdm(haver.create_dataset()), batch_size):
        toks = haver.model.to_tokens(texts)
        all_tokens = [t for tok in toks for t in tok]
        proc = sum(map(len, toks))
        all_token_ids = [tokens_viewed + i for i in range(proc)]
        for i, t in enumerate(all_token_ids):
            if t in tokens:
                activ = activs[tokens.index(t)]
                if activ < thresh:
                    continue
                print(activ, repr(haver.model.decode(all_tokens[max(0, i - sli + 1):i+1])),
                      repr(haver.model.decode(all_tokens[i+1:i+5])))
        tokens_viewed += proc
        if tokens_viewed > max(tokens):
            break
for i in range(10_000, 10_0100):
    visualize(i)

0.0016378754550970874


0it [00:00, ?it/s]

2024-05-11 20:53:21.308701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


8.765625 'split into multiple parts that<s><|assistant|> The ratio of the radius of the smaller circle to that of the larger circle is $\\' 'frac{1}{'
8.5234375 'long.  Thus, the ratio of the area of the smaller circle to that of the larger circle is $\\left(\\' 'frac{1}{'
9.265625 '0$, we have by the AM-GM inequality that\n\\[\\sqrt[3]{xyz} \\le \\' 'frac{x+'
8.0625 '\n\\[\\sqrt[3]{xyz} \\le \\frac{x+y+z}{3} = \\' 'frac{7}{'
8.9296875 'x+y+z}{3} = \\frac{7}{3}.\\]Thus $xyz \\le \\' 'frac{7<s>'
10.0546875 'following formula for a (0,2)-tensor:\n\n$$\\nabla_k T_{ij} = \\' 'frac{\\partial T'
6.9296875 'formula for a (0,2)-tensor:\n\n$$\\nabla_k T_{ij} = \\frac' '{\\partial T_{'
9.6328125 '<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><s><|assistant|> No;<|end|><s><|assistant|> By the Angle Bisector Theorem,\n\\[\\' 'frac{BD}{'
7.40234375 '<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><s><|assistant|> No;<|end|><s><|assistant|> By the Ang

0it [00:00, ?it/s]

6.26171875 '<s><|assistant|> Testing in Python can be done using the `unittest` module. You can mock `stdin`,' 'file inputs, and'
6.17578125 'CSS framework for its layout and design. \n\nThe first thing you need to do is to create a new CSS' 'file in your static'
6.7109375 'solution for aggregating and merging the data using Python:\n\n```python\ndef merge_data(name_' 'file, birthdates'
7.00390625 ', identify patterns in data, and make predictions and decisions.\n\nIn A<s><|assistant|> Here is a JSON data' 'file that contains a'
6.1015625 'to make a change in the `config/environments/production.rb` file of your Rails application. Open the' 'file and locate the'
0.023285800970873786


0it [00:00, ?it/s]

6.07421875 "<s><|assistant|> Alright, let's think about it together. We want to find out which one causes more matter (" 'like air) to'
6.18359375 '32-bit boundaries, so the value of 2 is correct.\n2. The documentation you provided, specifically' 'the Procedure Call'
6.43359375 'cylindrical coordinates. This system is particularly useful for solving problems related to cylindrical and symmetrical shapes, such' 'as cylinders'
6.0546875 "ruling party or coalition. The Prime Minister's role is to lead the executive branch of the government, which" 'includes the Council of'
7.7421875 "of change and constancy.<s><|assistant|> Liouville's theorem states that every bounded entire function (i.e" '., a function that'
6.11328125 'a heterogeneous system. In a heterogeneous catalytic reaction, the reactants are in a different phase (us' 'ually solid) than'
6.69140625 '<|assistant|> To find the perimeter of the trapezium, we need to find the lengths of the other two sides (' 'the non-parallel'
6.54

KeyboardInterrupt: 

In [10]:
haver.sae.push_to_hub("nev/phi-3-4k-saex-test", "l11-test0-l0-60")

Uploading config...
Saving weights...
Uploading weights...


In [ ]:
%load_ext autoreload
%autoreload 2
from scripts.train_phi_sae import main
main(layer=11, is_gated=True, sparsity_coefficient=1.4e-5, n_devices=4)

2024-05-11 18:35:48.460222: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading model...


/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Creating SAE...
Loading dataset...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Creating buffer...
Training for 100000 iterations
Sparsity coefficient: 1.4e-05


wandb: Currently logged in as: neverix. Use `wandb login --relogin` to force relogin


Learning rate: 0.0004 warmed up for 128 iterations and cycled every 100000 iterations


  0%|          | 0/100000 [00:00<?, ?it/s]

buf 0.0
49170.3671875
0.0
buf 24.585186004638672
49169.7109375
4917.037109375
buf 49.1700439453125
48581.9765625
9342.3046875
buf 73.4610366821289
47364.4140625
13266.271484375
buf 97.14324188232422
45614.1796875
16676.0859375
buf 119.95034790039062
43386.0859375
19569.89453125
buf 141.64337158203125
41042.234375
21951.51171875
buf 162.16448974609375
38310.20703125
23860.5859375
buf 181.31961059570312
35663.01171875
25305.546875
buf 199.15109252929688
33010.59375
26341.29296875
buf 215.65640258789062
30800.330078125
27008.22265625
buf 231.05654907226562
28479.984375
27387.4296875
buf 245.29656982421875
26396.130859375
27496.6875
buf 258.49462890625
24632.990234375
27386.62890625
buf 270.81109619140625
23329.203125
27111.265625
buf 282.4757080078125
21993.94140625
26733.0625
buf 293.47271728515625
21034.201171875
26259.146484375
buf 303.98980712890625
19922.357421875
25736.65234375
buf 313.9509582519531
19120.798828125
25155.22265625
buf 323.51141357421875
18148.45703125
24551.77734375


In [ ]:
from scripts.train_gated_sae import main
main(cache_batch_size=256)

In [ ]:
from scripts.train_phi_sae import main
main(layer=11, is_gated=True)

In [ ]:
from scripts.train_phi_sae import main
main(layer=11, is_gated=True, sparsity_coefficient=4.2e-6)

In [ ]:
from scripts.train_phi_sae import main
main(layer=30)

In [ ]:
from scripts.train_gated_sae import main
main(is_xl=True, layer=20, cache_batch_size=256)
# main(is_xl=True, layer=30, cache_batch_size=256, restore="weights/gpt2-20-gated.safetensors")

In [ ]:
from scripts.train_gpt2_sae import main
main(restore="weights/gpt2-20-base.safetensors", layer=32)
# python -m scripts.train_gpt2_sae --save_steps 0 --is_xl=False --layer=9
# main(save_steps=0, is_xl=False, layer=1)